# Basic summary of what this capstone will, and may accomplish

* Scrape data from indeed, and return job postings
* properly format location, title, requirements
* analyze what words and requirements occur the most
* Determine the quality of each job posting (how generic does it sound, and how common are the phrases that they are using) - this is important, but needs a bit of refinement

Stretch goals
* add input to allow users to specifiy the cities to look at

* add a map to visualize how many postings there are in each city 

Current Issues
* Sponsored job posts show up in data multiple times, and location data is missing
* Summary page is not formated properly, .split() doesn't fix it

# Indeed Job Scraper

So far, it can scrape a job's

* location
* title
* company name
* salary
* summary

to-do

* responsibilities
* requirements

not required, but would be nice

* additional queries
* additional cities

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

In [2]:
max_results_per_city = 100
#city_set = ['New+York','Chicago','San+Francisco', 'Austin', 'Seattle', 'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Washington+DC', 'Boulder']
# titles = ['data+analyst', 'business+analyst', 'data+scientist', 'machine+learning']
city_set = ['New+York']
columns = ['city', 'job_title', 'company_name', 'location', 'summary']
sample_df = pd.DataFrame(columns = columns)

In [3]:
def parse(query,metro):
    
    # Target url, and n number of pages to scrape
    url = "https://www.indeed.com/jobs?q={}&l={}&start={}"
    n = 30
    
    # Create dataframe that we'll return later
    df = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis", "Query", "Metro"])
    
    # Scrape first n pages
    for i in range(0,n):
        html = requests.get(url.format(query, metro, i))
        time.sleep(1)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobtitle').text.replace('\n', '')
            except:
                title = 'None'
            try:
                location = each.find('span', {'class':"location" }).text.replace('\n', '')
            except:    
                try:
                    location = each.find('div', {'class':"location" }).text.replace('\n', '')
                except:
                    location = 'None'
            try: 
                company = each.find(class_='company').text.replace('\n', '')
            except:
                company = 'None'
            try:
                salary = each.find('span', {'class':'no-wrap'}).text
            except:
                salary = 'None'
            synopsis = each.find('span', {'class':'summary'}).text.replace('\n', '')
            df = df.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis, 'Query': query, 'Metro': metro}, ignore_index=True)
    return df

In [4]:
# Lists of positions and locations to search
positions = ['data+analyst', 'data+scientist', 'data+engineer']
cities = ['washington+dc']

# Create the dataframe
data = pd.DataFrame()

# Scrape all positions in all cities specified, and append to dataframe
for position in positions:
    for city in cities:
        data = pd.concat([data, parse(position, city)])

In [5]:
# Save scraped data to a local directory
if os.path.exists("data/dc_indeed.csv"):
    os.remove("data/dc_indeed.csv")
data.to_csv('data/dc_indeed.csv', encoding='utf-8')

In [6]:
# Preview the data
data.head(10)

,Title,Location,Company,Salary,Synopsis,Query,Metro
0,Data Analyst,"Washington, DC",Chenega Corporation,None,Perform data entry...,data+analyst,washington+dc
1,Program Data Analyst,"Arlington, VA",RGS,None,Organize and build...,data+analyst,washington+dc
2,Data Scientist - Human Capital Analytics,"Bethesda, MD 20816",Leidos,None,"Review, assess dat...",data+analyst,washington+dc
3,Intelligence Analyst with TS/SCI,"Washington, DC",Object CTalk Inc.,None,Strong analytical ...,data+analyst,washington+dc
4,Systems Analyst/Data Scientist,"Springfield, VA",Radiant Solutions,None,We build advanced ...,data+analyst,washington+dc
5,Aviation Data Systems Analyst,"Washington, DC",Concepts Beyond,None,The Analyst will p...,data+analyst,washington+dc
6,Health Interventions and Claims Data Analyst,"Washington, DC 20007 (Georgetown area)",Innovation Center for Biomedical Infor...,None,We are working on applying data sc...,data+analyst,washington+dc
7,Political Data Service Analyst,"Washington, DC 20005 (Logan Circle area)",Data Trust,None,The Data Services Analyst will be ...,data+analyst,washington+dc
8,Part Time/Full Time Administrative Assistant,"Washington, DC 20003 (Capitol Hill area)",General Dynamics Information Technology,\n $16 - $18 an hour,Support Program + Management analy...,data+analyst,washington+dc
9,Contractual Data Analyst (STASI),"Washington, DC 20431 (Foggy Bottom area)",International Monetary Fund,None,Contractual Data Analyst (STASI) (...,data+analyst,washington+dc


In [7]:
# Return size of dataframe
data.shape

(1485, 7)